# 05 - ReAct Agents

**Build agents that reason and act!** Learn the ReAct pattern - the foundation of modern AI agents.

## Learning Objectives

By the end of this notebook, you will:
- Understand the ReAct (Reasoning + Acting) framework
- Implement the Thought → Action → Observation loop
- Build a ReAct agent from scratch
- Know when to stop and return results

## Table of Contents

1. [What is ReAct?](#what-is-react)
2. [The ReAct Loop](#loop)
3. [Building a ReAct Agent](#building)
4. [Using Our Framework](#framework)
5. [Exercises](#exercises)
6. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
import json
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

from openai import OpenAI
client = OpenAI()

print("Setup complete!")

---
## 1. What is ReAct? <a id='what-is-react'></a>

**ReAct** = **Re**asoning + **Act**ing

It's a prompting pattern where the LLM:
1. **Thinks** about what to do (Thought)
2. **Acts** by calling a tool (Action)
3. **Observes** the result (Observation)
4. **Repeats** until the task is complete

### Why ReAct Works

- **Transparency**: We see the agent's reasoning
- **Grounding**: Actions are based on real observations
- **Flexibility**: Can adapt to unexpected results
- **Reliability**: Step-by-step reduces errors

In [ ]:
# GUIDED: Example of ReAct reasoning

example = """
Question: What is the population of the capital of France?

Thought: I need to find the capital of France first, then look up its population.
Action: search
Action Input: {"query": "capital of France"}
Observation: Paris is the capital of France.

Thought: Now I know Paris is the capital. I need to find Paris's population.
Action: search
Action Input: {"query": "population of Paris"}
Observation: Paris has a population of approximately 2.1 million in the city proper.

Thought: I now have the answer. The capital of France is Paris with about 2.1 million people.
Action: finish
Action Input: {"answer": "The capital of France is Paris, with a population of approximately 2.1 million."}
"""

print(example)

---
## 2. The ReAct Loop <a id='loop'></a>

```
┌─────────────────────────────────────────┐
│                  START                   │
└─────────────────┬───────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────┐
│   THOUGHT: Reason about what to do      │
└─────────────────┬───────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────┐
│   ACTION: Choose and execute a tool     │
└─────────────────┬───────────────────────┘
                  │
          ┌───────┴───────┐
          │  Is action    │
          │  'finish'?    │
          └───────┬───────┘
           No │         │ Yes
              │         │
              ▼         ▼
┌────────────────┐  ┌─────────────────────┐
│ OBSERVATION:   │  │       DONE          │
│ Get result     │  │  Return answer      │
└───────┬────────┘  └─────────────────────┘
        │
        └──────────────┐
                       │
            (loop back to THOUGHT)
```

In [ ]:
# GUIDED: Define tools for our agent

def search(query: str) -> str:
    """Mock search function."""
    # Simulated search results
    results = {
        "capital of france": "Paris is the capital of France.",
        "population of paris": "Paris has a population of approximately 2.1 million.",
        "python": "Python is a high-level programming language.",
        "weather": "Weather varies by location and season."
    }
    
    query_lower = query.lower()
    for key, value in results.items():
        if key in query_lower:
            return value
    
    return f"Found information about: {query}"

def calculator(expression: str) -> str:
    """Safe calculator."""
    try:
        allowed = set('0123456789+-*/.() ')
        if all(c in allowed for c in expression):
            return str(eval(expression))
    except:
        pass
    return "Error: Invalid expression"

TOOLS = {
    "search": search,
    "calculator": calculator
}

print("Tools available:", list(TOOLS.keys()))

---
## 3. Building a ReAct Agent <a id='building'></a>

Let's build a ReAct agent from scratch!

In [ ]:
# GUIDED: The ReAct prompt template

REACT_PROMPT = """You are a helpful AI assistant that solves problems step by step.

You have access to these tools:
- search(query): Search for information on any topic
- calculator(expression): Perform math calculations

For each step, respond in this exact format:

Thought: [Your reasoning about what to do next]
Action: [tool name]
Action Input: {{"param": "value"}}

When you have the final answer, use:

Thought: [Why you're ready to answer]
Action: finish
Action Input: {{"answer": "Your final answer"}}

Always think before acting. Be concise but thorough.
"""

print("ReAct prompt template defined!")

In [ ]:
# GUIDED: Parse the LLM's response

def parse_react_response(response: str) -> dict:
    """
    Parse a ReAct-formatted response.
    
    Returns dict with 'thought', 'action', 'action_input'
    """
    result = {
        "thought": "",
        "action": "",
        "action_input": {}
    }
    
    lines = response.strip().split("\n")
    
    for i, line in enumerate(lines):
        line = line.strip()
        
        if line.lower().startswith("thought:"):
            result["thought"] = line[8:].strip()
            
        elif line.lower().startswith("action:"):
            result["action"] = line[7:].strip()
            
        elif line.lower().startswith("action input:"):
            input_str = line[13:].strip()
            # Try to find JSON in this line or subsequent lines
            try:
                result["action_input"] = json.loads(input_str)
            except:
                # Look for JSON in remaining text
                remaining = "\n".join(lines[i:])
                start = remaining.find("{")
                end = remaining.rfind("}") + 1
                if start >= 0 and end > start:
                    try:
                        result["action_input"] = json.loads(remaining[start:end])
                    except:
                        result["action_input"] = {"raw": input_str}
    
    return result

# Test the parser
test_response = """Thought: I need to search for the capital of France.
Action: search
Action Input: {"query": "capital of France"}"""

print("Parsed:", parse_react_response(test_response))

In [ ]:
# GUIDED: The main ReAct loop

def run_react_agent(question: str, max_steps: int = 5, verbose: bool = True) -> str:
    """
    Run a ReAct agent on a question.
    """
    messages = [
        {"role": "system", "content": REACT_PROMPT},
        {"role": "user", "content": f"Question: {question}"}
    ]
    
    for step in range(1, max_steps + 1):
        if verbose:
            print(f"\n{'='*50}")
            print(f"Step {step}")
            print('='*50)
        
        # Get LLM response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        
        content = response.choices[0].message.content
        if verbose:
            print(f"\nLLM Response:\n{content}")
        
        # Parse the response
        parsed = parse_react_response(content)
        
        if verbose:
            print(f"\nParsed:")
            print(f"  Thought: {parsed['thought']}")
            print(f"  Action: {parsed['action']}")
            print(f"  Input: {parsed['action_input']}")
        
        # Check for finish
        if parsed["action"].lower() == "finish":
            answer = parsed["action_input"].get("answer", str(parsed["action_input"]))
            if verbose:
                print(f"\n{'='*50}")
                print(f"FINAL ANSWER: {answer}")
                print('='*50)
            return answer
        
        # Execute the action
        action = parsed["action"].lower()
        if action in TOOLS:
            # Get the first value from action_input for simple tools
            input_values = list(parsed["action_input"].values())
            if input_values:
                observation = TOOLS[action](input_values[0])
            else:
                observation = "Error: No input provided"
        else:
            observation = f"Error: Unknown tool '{action}'"
        
        if verbose:
            print(f"\nObservation: {observation}")
        
        # Add to messages for next iteration
        messages.append({"role": "assistant", "content": content})
        messages.append({"role": "user", "content": f"Observation: {observation}"})
    
    return "Max steps reached without finding answer"

In [ ]:
# GUIDED: Test the agent!

answer = run_react_agent("What is the population of the capital of France?")

In [ ]:
# GUIDED: Try a calculation question

answer = run_react_agent("What is 15 * 23 + 47?")

---
## 4. Using Our Framework <a id='framework'></a>

Now let's use the `ReActAgent` from our framework.

In [ ]:
# GUIDED: Use our ReActAgent class

from src.llm_client import LLMClient
from src.tool_registry import Tool, ToolRegistry
from src.agent_framework import ReActAgent, create_react_agent

# Create tools
registry = ToolRegistry()

registry.register(Tool(
    name="search",
    description="Search for information on any topic",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "Search query"}
        },
        "required": ["query"]
    },
    function=search
))

registry.register(Tool(
    name="calculator",
    description="Perform mathematical calculations",
    parameters={
        "type": "object",
        "properties": {
            "expression": {"type": "string", "description": "Math expression"}
        },
        "required": ["expression"]
    },
    function=calculator
))

# Create agent
llm = LLMClient(provider="openai", model="gpt-4o-mini")
agent = ReActAgent(
    name="research_agent",
    llm=llm,
    tools=registry,
    max_steps=5,
    verbose=True
)

print("Agent created!")

In [ ]:
# GUIDED: Run the agent

result = agent.run("What is Python and when was it created?")

print("\n" + "="*50)
print("Result Summary:")
print(f"  Success: {result.success}")
print(f"  Steps: {len(result.steps)}")
print(f"  Output: {result.output}")

---
## 5. Exercises <a id='exercises'></a>

### Exercise 1: Add a New Tool

Add a `get_time` tool and test the agent with it.

In [ ]:
# TODO: Add a get_time tool that returns the current time
# Then ask the agent "What time is it?"

# Your code here:


### Exercise 2: Multi-Step Problem

Create a question that requires multiple tool uses.

In [ ]:
# TODO: Ask a question that requires 2+ tool calls
# Example: "What is 10% of the population of Paris?"

# Your code here:


### Exercise 3: Error Handling

Test what happens when the agent encounters an error.

In [ ]:
# TODO: Ask a question that might cause an error
# See how the agent handles it

# Your code here:


---
## 6. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You understand the ReAct pattern (Thought → Action → Observation)
- [ ] You can implement a basic ReAct loop
- [ ] You know how to parse agent responses
- [ ] You can use our ReActAgent framework
- [ ] You completed at least 2 exercises

### Next Steps

In the next notebook, we'll explore **Multi-Agent Systems** - coordinating multiple agents to solve complex problems!

---
## Summary

**ReAct Pattern:**
1. **Thought**: Reason about what to do
2. **Action**: Execute a tool
3. **Observation**: See the result
4. **Repeat** until done

**Key Components:**
- System prompt with format instructions
- Response parser
- Tool executor
- Loop controller

**Best Practices:**
- Set a max step limit
- Handle errors gracefully
- Log all steps for debugging
- Use low temperature for consistency